In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications import Xception
from tensorflow import keras
import tensorflow as tf
import os
from PIL import Image, ImageOps
from tqdm import tqdm
import numpy as np

In [2]:
X = Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3)
)

for layer in X.layers:
    layer.trainable = False

In [3]:
x = Flatten()(X.output)
X_out = Dense(2, activation='softmax')(x)

In [4]:
from keras.models import Model
model = Model(inputs=X.input, outputs=X_out)
#model.summary()

In [5]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.02),
    metrics=['accuracy'],
)

In [6]:
# Collecting Train Data

DATA_PATH = "../../my-datasets/Hurricane_Damage/train_another/"
CATEGORIES = ["damage", "no_damage"]

train_data_list = []

for category in CATEGORIES:
    dir_path = os.path.join(DATA_PATH, category)
    img_list = os.listdir(dir_path)
    class_num = CATEGORIES.index(category)
    print(f"Collecting from train_another/{category}/")
    for img_path in tqdm(img_list):
        img = Image.open(os.path.join(dir_path, img_path))
        img_arr = np.array(img, dtype=int)
        if img_arr.shape[-1] == 4:
            img_arr = img_arr[...,:3]
        if img_arr.shape != (128, 128, 3):
            continue
        train_data_list.append([img_arr, class_num])
        
print(f"Total number of samples is {len(train_data_list)}")

100%|██████████| 5000/5000 [00:05<00:00, 934.56it/s]


100%|██████████| 5400/5400 [00:06<00:00, 835.22it/s]

Total number of samples is 10361


In [7]:
import random
random.shuffle(train_data_list)

X_train = []
y_train = []

for image, label in train_data_list:
    X_train.append(image)
    y_train.append(label)

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print(X_train.shape)
print(y_train.shape)

### Converting y into a one-hot vector ###
shape = (y_train.size, y_train.max()+1)
y_train_one_hot = np.zeros(shape)
rows = np.arange(y_train.size)
y_train_one_hot[rows, y_train] = 1

print(y_train_one_hot.shape)

(10361, 128, 128, 3)
(10361,)
(10361, 2)


In [8]:
# Collecting Test Data

DATA_PATH = "../../my-datasets/Hurricane_Damage/test/"
CATEGORIES = ["damage", "no_damage"]

test_data_list = []

for category in CATEGORIES:
    dir_path = os.path.join(DATA_PATH, category)
    img_list = os.listdir(dir_path)
    class_num = CATEGORIES.index(category)
    print(f"Collecting from test/{category}/")
    for img_path in tqdm(img_list):
        img = Image.open(os.path.join(dir_path, img_path))
        img_arr = np.array(img)
        test_data_list.append([img_arr, class_num])
        
print(f"Total number of samples is {len(test_data_list)}")

100%|██████████| 1000/1000 [00:00<00:00, 1066.54it/s]


100%|██████████| 1000/1000 [00:00<00:00, 1237.18it/s]

Total number of samples is 2000


In [9]:
import random
random.Random(30).shuffle(test_data_list)

X_test = []
y_test = []

for image, label in test_data_list:
    X_test.append(image)
    y_test.append(label)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print(X_test.shape)
print(y_test.size)

### Converting y_test into a one-hot vector ###
shape = (y_test.size, y_test.max()+1)
y_test_one_hot = np.zeros(shape)
rows = np.arange(y_test.size)
y_test_one_hot[rows, y_test] = 1

(2000, 128, 128, 3)
2000


In [10]:
from keras.callbacks import ModelCheckpoint

checkpoint_path = "archive/Xception_model/Xception_damage_classification.ckpt"
callbacks_list = [
    ModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        save_weights_only=True
    ),
]

# This line below must be commented when running this cell for the first time
#model.load_weights(checkpoint_path)

model_history = model.fit(
    x=X_train,
    y=y_train_one_hot,
    validation_data=(X_test, y_test_one_hot),
    batch_size=8,
    epochs=15,
    verbose=1,
    callbacks=callbacks_list
) 

2022-02-03 13:51:55.630533: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4074110976 exceeds 10% of free system memory.
2022-02-03 13:51:59.572122: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4074110976 exceeds 10% of free system memory.


Epoch 1/15
1296/1296 [==============================] - 30s 18ms/step - loss: 233.3479 - accuracy: 0.7685 - val_loss: 234.0477 - val_accuracy: 0.7530
Epoch 2/15
1296/1296 [==============================] - 22s 17ms/step - loss: 229.9109 - accuracy: 0.8071 - val_loss: 142.9180 - val_accuracy: 0.8415
Epoch 3/15
1296/1296 [==============================] - 21s 16ms/step - loss: 197.3234 - accuracy: 0.8278 - val_loss: 147.0056 - val_accuracy: 0.8400
Epoch 4/15
1296/1296 [==============================] - 21s 16ms/step - loss: 196.2773 - accuracy: 0.8399 - val_loss: 461.0207 - val_accuracy: 0.7205
Epoch 5/15
1296/1296 [==============================] - 21s 16ms/step - loss: 176.9405 - accuracy: 0.8512 - val_loss: 308.2006 - val_accuracy: 0.7870
Epoch 6/15
1296/1296 [==============================] - 22s 17ms/step - loss: 160.7244 - accuracy: 0.8581 - val_loss: 126.6076 - val_accuracy: 0.8860
Epoch 7/15
1296/1296 [==============================] - 22s 17ms/step - loss: 158.0875 - accuracy: 0

In [11]:
model_history.history

{'loss': [233.34791564941406,
  229.91091918945312,
  197.32342529296875,
  196.27734375,
  176.9404754638672,
  160.7244415283203,
  158.08750915527344,
  143.9647674560547,
  181.1988525390625,
  148.7984161376953,
  130.26416015625,
  155.103515625,
  123.53437042236328,
  127.93260955810547,
  116.9615249633789],
 'accuracy': [0.7684586644172668,
  0.807064950466156,
  0.82781583070755,
  0.8398803472518921,
  0.851172685623169,
  0.8581218123435974,
  0.8640092611312866,
  0.8703793287277222,
  0.8684489727020264,
  0.8806099891662598,
  0.8859183192253113,
  0.8796448111534119,
  0.8940256834030151,
  0.8931570053100586,
  0.8997201323509216],
 'val_loss': [234.0476531982422,
  142.9180145263672,
  147.00559997558594,
  461.0207214355469,
  308.2005920410156,
  126.60762023925781,
  366.19171142578125,
  622.20947265625,
  136.9326629638672,
  162.8605194091797,
  144.76861572265625,
  202.83786010742188,
  436.649658203125,
  777.0240478515625,
  150.12408447265625],
 'val_accur

In [ ]:
checkpoint_path = "archive/Xception_model/Xception_damage_classification.ckpt"

X = Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3)
)

x = Flatten()(X.output)
X_out = Dense(2, activation='softmax')(x)
new_model = Model(inputs=X.input, outputs=X_out)

new_model.load_weights(checkpoint_path)

In [ ]:
path = "../../my-datasets/Hurricane_Damage/test/damage/-93.65936500000001_30.138793.jpeg"
test_img = Image.open(path)
test_arr = np.asarray(test_img)
test_arr = np.expand_dims(test_arr, axis=0) 
pred = new_model.predict(x=test_arr)
print(pred)
test_img

In [ ]:
pred_ = np.argmax(pred, axis=1)
if(pred_ == 0):
    print("Hello")